# Analiza podobnych dokumentów

## Zacznijmy od importu potrzebnych bibiotek

In [ ]:
import pandas as pd
import numpy as np
import sklearn.feature_extraction.text as txt
from scipy.spatial.distance import euclidean
from sklearn.neighbors import NearestNeighbors

## Załadujmy dane na temat różnych ludzi

In [ ]:
people = pd.read_csv('dane/people_wiki.csv')

## Przyjrzyjmy się tym danym

In [ ]:
people.head()

## Wybierz swoją postać

### Kilka przykładowych postaci

In [ ]:
gates = people[people['name'] == 'Bill Gates']
clooney = people[people['name'] == 'George Clooney']
therock = people[people['name'] == 'Dwayne Johnson']

my_celeb = therock

### Życiorys wybranej postaci

In [ ]:
my_celeb.text.iloc[0]

## Policzmy słowa w artykule nt. wybranej postaci

In [ ]:
my_vect = txt.CountVectorizer()
my_counts = my_vect.fit_transform(my_celeb.text)
my_counts_df = pd.DataFrame.sparse.from_spmatrix(my_counts).T
my_counts_df.rename(columns={0: 'count'}, inplace=True)
my_counts_df['word'] = pd.DataFrame(my_vect.get_feature_names_out())
my_counts_df = my_counts_df[my_counts_df['count'] != 0.]
my_counts_df.sort_values(by='count', ascending=False, inplace=True)
my_counts_df.head(10)

## Obliczmy TF-IDF dla całego zbioru artykułów

In [ ]:
tfidf_vect = txt.TfidfVectorizer()
matrix = tfidf_vect.fit_transform(people.text)

## Zbadajmy TF-IDF dla wybranej postaci

In [ ]:
tfidf_celeb = tfidf_vect.transform(my_celeb.text)
tfidf_celeb_df = pd.DataFrame.sparse.from_spmatrix(tfidf_celeb).T
tfidf_celeb_df.rename(columns={0: 'tfidf'}, inplace=True)
tfidf_celeb_df['word'] = pd.DataFrame(tfidf_vect.get_feature_names_out())
tfidf_celeb_df = tfidf_celeb_df[tfidf_celeb_df['tfidf'] != 0.]
tfidf_celeb_df.sort_values(by='tfidf', ascending=False, inplace=True)
tfidf_celeb_df.head(10)

## Zmierzmy dystans między tekstami opisującymi różne postacie

### Kilka przykładowych postaci

In [ ]:
taylor = people[people['name'] == 'Taylor Swift']
alicia = people[people['name'] == 'Alicia Keys']
therock = people[people['name'] == 'Dwayne Johnson']

celeb_1 = taylor
celeb_2 = alicia

### Wektoryzacja celebrytów

In [ ]:
tfidf_celeb_1 = tfidf_vect.transform(celeb_1.text).toarray()
tfidf_celeb_2 = tfidf_vect.transform(celeb_2.text).toarray()

### Dystans między celebrytami

In [ ]:
euclidean(tfidf_celeb_1, tfidf_celeb_2)

## Algorytm najbliższych sąsiadów

### Zbudujmy model

In [ ]:
knn_model = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(matrix)

### Sprawdźmy, kto jest najbardziej podobny do wybranej postaci

#### Eksperymentuj z różnymi postaciami

In [ ]:
name = 'Taylor Swift'
# 'Angelina Jolie', 'Arnold Schwarzenegger', 'Tom Hardy'

In [ ]:
row = people[people.name==name].index.tolist()[0]
closest_friends = knn_model.kneighbors(matrix.getrow(row), return_distance=True)
names_index = closest_friends[1][0]
names = [people.iloc[row]['name'] for row in names_index]
distances = closest_friends[0][0]
pd.DataFrame({'Distances': distances,'Neighbours':names })